In [1]:
import tensorflow as tf

def train():
    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    print("Training...")

    model.fit(
        x_train, y_train, 
        epochs=3, 
        validation_split=0.2 
    ) 

    score = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
    print('Test accuracy: ', score[1])

In [2]:
# Local training
train()

AttributeError: 'NoneType' object has no attribute 'keras'

In [3]:
import os
from kubeflow import fairing

PRIVATE_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'

fairing.config.set_builder(
    'append',
    # base_image = f'{PRIVATE_REGISTRY}/kf-base:latest', # 사전준비에서 마련한 Base Image  # 동작 (X) 하단 설명으로 이슈 해결할 것
    base_image= 'tensorflow/tensorflow:2.0.3-gpu-py3', # 동작 (O)
    registry = PRIVATE_REGISTRY, 
    push=True
)

fairing.config.set_deployer('job')

remote_train = fairing.config.fn(train)

remote_train()

[I 201207 00:13:37 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.function.FunctionPreProcessor object at 0x7f4d207eab38>
[I 201207 00:13:37 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f4d1bd0a9e8>
[I 201207 00:13:37 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7f4d207eabe0>
[W 201207 00:13:37 append:50] Building image using Append builder...
[I 201207 00:13:37 base:107] Creating docker context: /tmp/fairing_context_6_2l7nno
[W 201207 00:13:37 base:94] /usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 201207 00:13:37 docker_creds_:234] Loading Docker credentials for repository 'tensorflow/tensorflow:2.0.3-gpu-py3'
[W 201207 00:13:39 append:54] Image successfully built in 2.4830146699996476s.
[W 201207 00:13:39 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/fairing-job:68EA1BE0...
[I 201207 

Traceback (most recent call last):
  File "/app/function_shim.py", line 79, in <module>
    call(args.serialized_fn_file)
  File "/app/function_shim.py", line 64, in call
    res = obj()
  File "<ipython-input-1-0dbc804d7d9a>", line 4, in train
AttributeError: 'NoneType' object has no attribute 'keras'


### curl로 프라이빗 레지스트리 확인

In [ ]:
! curl http://kubeflow-registry.default.svc.cluster.local:30000/v2/_catalog

In [ ]:
! curl http://kubeflow-registry.default.svc.cluster.local:30000/v2/fairing-job/tags/list

# Function Fairing 패키지 버전 이슈
Function Fairing 시 fairing SDK가 컨테이너 내부에서 아래와 같은 명령을 실행
```python
python /app/function_shim.py --serialized_fn_file /app/pickled_fn.p --python_version 3.6
```
결과는 에러.

다음과 같이 테스트 하여 실행 성공과 실패 케이스를 정리함

- 실행 실패
  - base_image = f'{PRIVATE_REGISTRY}/kf-base:latest', # 사전준비에서 마련한 Base Image  
  - base_image = 'tensorflow/tensorflow:latest-py3',   
  - base_image = 'tensorflow/tensorflow:2.1.2-gpu',    
  - Base Image에 pip install tensorflow-gpu   <- 최신 2.3.1 설치됨
- 실행 성공  
  - base_image = 'brightfly/kubeflow-jupyter-lab:tf2.0-gpu', <- 실마리
  - base_image = 'tensorflow/tensorflow:2.0.3-gpu-py3',
  - Base Image에 pip install tensorflow-gpu==2.0.0 